In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Analyzing CIA Factbook Data Using SQL

We'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth.

we will gather info from the CD about the name of the table and what the table looks like

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We now run another query that returns the first five rows of the facts table in the database.

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


We will now write a query that returns the following:

Minimum population
Maximum population
Minimum population growth
Maximum population growth

In [4]:
%%sql
SELECT MIN (population), MAX (population), MIN (population_growth), MAX (population_growth)
  FROM facts;

Done.


MIN (population),MAX (population),MIN (population_growth),MAX (population_growth)
0,7256490011,0.0,4.02


We see a country with 0 population and another with 7 billion. We will write queries to understand which countries have these numbers

In [5]:
%%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE population = (SELECT MIN(population)
                             FROM facts);

Done.


COUNTRY
Antarctica


In [6]:
 %%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE population = (SELECT MAX(population)
                             FROM facts);

Done.


COUNTRY
World


We will now write a new query excluding the World that returns the following:

Minimum population
Maximum population
Minimum population growth
Maximum population growth

In [7]:
%%sql
SELECT MIN (population), MAX (population), MIN (population_growth), MAX (population_growth)
  FROM facts
    WHERE name != 'World';

Done.


MIN (population),MAX (population),MIN (population_growth),MAX (population_growth)
0,1367485388,0.0,4.02


We are curious on what are the average of population and area

In [8]:
%%sql
SELECT AVG (population), AVG (area)
  FROM facts
    WHERE name != 'World';

Done.


AVG (population),AVG (area)
32242666.56846473,555093.546184739


We write a new query that finds all countries meeting both of the following criteria:

The population is above average and the area is below average.

We will order by area to understand which are smaller

In [9]:
%%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE population > (SELECT AVG (population)
  FROM facts
    WHERE name != 'World') AND area < (SELECT AVG (area)
  FROM facts
    WHERE name != 'World') 
    ORDER BY area;

Done.


COUNTRY
"Korea, South"
Bangladesh
Uganda
United Kingdom
Philippines
Italy
Poland
Vietnam
Germany
Japan


Which country has the most people?

In [10]:
%%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE population = (SELECT MAX(population)
  FROM facts
    WHERE name != 'World');

Done.


COUNTRY
China


Which country has the highest growth rate

In [11]:
%%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE population_growth = (SELECT MAX(population_growth)
  FROM facts
    WHERE name != 'World');

Done.


COUNTRY
South Sudan


Which countries will add the most people to their populations next year?

In [12]:
%%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE name != 'World'
    ORDER BY population_growth * population DESC
    LIMIT 5;

Done.


COUNTRY
India
China
Nigeria
Pakistan
Ethiopia


Which countries have a higher death rate than birth rate?

In [13]:
%%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE birth_rate < death_rate
    LIMIT 5;

Done.


COUNTRY
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia


Which countries have the highest population/area ratio?

In [14]:
%%sql
SELECT name AS COUNTRY, population/area AS population_to_area_ratio
  FROM facts
    ORDER BY population_to_area_ratio DESC
    LIMIT 5;

Done.


COUNTRY,population_to_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191


Which countries have the highest ratios of water to land?

In [15]:
%%sql
SELECT name AS COUNTRY, ROUND (CAST (area_water AS Float)/CAST (area_land AS Float),2) AS water_to_land_area_ratio
  FROM facts
    ORDER BY water_to_land_area_ratio DESC
    LIMIT 5;

Done.


COUNTRY,water_to_land_area_ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28


Which countries have more water than land?

In [16]:
%%sql
SELECT name AS COUNTRY
  FROM facts
    WHERE area_water > area_land
    LIMIT 5;

Done.


COUNTRY
British Indian Ocean Territory
Virgin Islands
